# Eval

- Metrike: Precision, Recall, F1, Accuracy (s TN) pri pragu t

- Rangirne metrike med pozitivnimi: Acc@1, P@5, MRR



In [60]:

# Parameters — prilagodi poti
PREDS_JSON = "notebook_velenje/result/velenje_match.json"
GOLD_JSON = "notebook_velenje/gold/trbovlje_to_velenje.json"
TOP_K = 5
THRESHOLD = 0.7
SCORE_FIELDS = ["combined_score", "score"]
RUN_TAG = "velenje"  # uporabi za imena izvozov


In [61]:

# Imports & helpers
import sys
from importlib import reload
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append("notebook_brezice")

import json_eval_helpers as H
reload(H)

print("Na voljo helper funkcije:", [n for n in dir(H) if not n.startswith("_")])


Na voljo helper funkcije: ['Dict', 'List', 'Optional', 'Set', 'Tuple', 'compute_metrics_from_maps', 'compute_open_world_metrics', 'decide_label_with_threshold', 'json', 'load_gold_json', 'load_predictions_json', 'load_predictions_with_scores', 'normalize']


In [62]:

# Naloži gold in predikcije
gold_map = H.load_gold_json(GOLD_JSON)

pred_pairs_map = H.load_predictions_with_scores(
    PREDS_JSON,
    score_field_options=SCORE_FIELDS,
    top_k=TOP_K
)

len(gold_map), len(pred_pairs_map)


(7, 7)

In [63]:

# Open-world metrike (odlocitve + rangiranje med pozitivnimi)
open_world = H.compute_open_world_metrics(
    gold_map,
    pred_pairs_map,
    threshold=THRESHOLD,
    k=TOP_K,
    exclude_no_match_from_mrr=True
)
open_world


{'decision_metrics': {'threshold': 0.7,
  'TP': 7,
  'FP': 0,
  'FN': 0,
  'TN': 0,
  'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'accuracy': 1.0},
 'ranking_metrics_among_positives': {'count_positives': 7,
  'acc_at_1': 1.0,
  'p_at_5': 0.19999999999999998,
  'mrr': 1.0},
 'details': [{'source': 'Naziv',
   'gold': ['Naziv'],
   'chosen': 'Naziv',
   'outcome': 'TP',
   'candidates': ['Naziv',
    'Plačljivo',
    'Cena',
    'Število parkirnih mest',
    'Lokacijska koordinata (GMAPS - WGS84)'],
   'scores': [0.9674472689628602,
    0.7569149570805686,
    0.7177644709746043,
    0.6801339772012498,
    0.6626261608941214]},
  {'source': 'Lokacijska koordinata (GMAPS - WGS84)',
   'gold': ['Lokacijska koordinata (GMAPS - WGS84)'],
   'chosen': 'Lokacijska koordinata (GMAPS - WGS84)',
   'outcome': 'TP',
   'candidates': ['Lokacijska koordinata (GMAPS - WGS84)',
    'Število parkirnih mest',
    'Število mest z električno polnilnico',
    'Cena',
    'Naziv'],
   'scores': [0.99

In [64]:

# Closed-world primerjava (Acc@1, P@5, MRR) — če helper to podpira
pred_map_labels = {s: [lab for (lab, _sc) in pairs] for s, pairs in pred_pairs_map.items()}
closed_metrics = H.compute_metrics_from_maps(gold_map, pred_map_labels, k=TOP_K)
closed_metrics


{'count': 7,
 'acc_at_1': 1.0,
 'p_at_5': 1.0,
 'mrr': 1.0,
 'details': [{'source_column': 'Naziv',
   'gold': ['Cena',
    'Lokacijska koordinata (GMAPS - WGS84)',
    'Naziv',
    'Plačljivo',
    'Število parkirnih mest'],
   'predictions': {'Naziv'},
   'hit': True,
   'rank_hit': 1,
   'hits_in_k': 1},
  {'source_column': 'Lokacijska koordinata (GMAPS - WGS84)',
   'gold': ['Cena',
    'Lokacijska koordinata (GMAPS - WGS84)',
    'Naziv',
    'Število mest z električno polnilnico',
    'Število parkirnih mest'],
   'predictions': {'Lokacijska koordinata (GMAPS - WGS84)'},
   'hit': True,
   'rank_hit': 1,
   'hits_in_k': 1},
  {'source_column': 'Plačljivo',
   'gold': ['Cena',
    'Električna polnilnica',
    'Plačljivo',
    'Število mest z električno polnilnico',
    'Število parkirnih mest'],
   'predictions': {'Plačljivo'},
   'hit': True,
   'rank_hit': 1,
   'hits_in_k': 1},
  {'source_column': 'Cena',
   'gold': ['Cena',
    'Električna polnilnica',
    'Naziv',
    'Plačlj

In [65]:

# Detajli po stolpcih + CSV izvoz
details_df = pd.DataFrame(open_world["details"])
display(details_df.head(20))

out_dir = Path("notebook_velenje/results_eval/results_eval_json")
out_dir.mkdir(parents=True, exist_ok=True)
out_csv = out_dir / f"eval_json_details_{RUN_TAG}.csv"
details_df.to_csv(out_csv, index=False, encoding="utf-8")
print("Saved CSV:", out_csv)


,source,gold,chosen,outcome,candidates,scores
0,Naziv,[Naziv],Naziv,TP,"[Naziv, Plačljivo, Cena, Število parkirnih mes...","[0.9674472689628602, 0.7569149570805686, 0.717..."
1,Lokacijska koordinata (GMAPS - WGS84),[Lokacijska koordinata (GMAPS - WGS84)],Lokacijska koordinata (GMAPS - WGS84),TP,"[Lokacijska koordinata (GMAPS - WGS84), Števil...","[0.9940255641937255, 0.6917464635129702, 0.680..."
2,Plačljivo,[Plačljivo],Plačljivo,TP,"[Plačljivo, Električna polnilnica, Število mes...","[0.9976082563400268, 0.7319330203533172, 0.689..."
3,Cena,[Cena],Cena,TP,"[Cena, Električna polnilnica, Plačljivo, Naziv...","[0.9774402379989624, 0.7157281847000121, 0.703..."
4,Število parkirnih mest,[Število parkirnih mest],Število parkirnih mest,TP,"[Število parkirnih mest, Število mest z elektr...","[0.9806933164596556, 0.7865170766567362, 0.691..."
5,Električna polnilnica,[Električna polnilnica],Električna polnilnica,TP,"[Električna polnilnica, Število mest z elektri...","[0.999554979801178, 0.8455830351302498, 0.7287..."
6,Število mest z električno polnilnico,[Število mest z električno polnilnico],Število mest z električno polnilnico,TP,"[Število mest z električno polnilnico, Elektri...","[0.9889903604984283, 0.8595469311663979, 0.784..."


Saved CSV: notebook_velenje/results_eval/results_eval_json/eval_json_details_velenje.csv


In [66]:
# Povzetek ključnih metrik
dm = open_world["decision_metrics"]  # to mora biti slovar!

if isinstance(dm, dict):
    print(f"Precision: {dm.get('precision', 0):.3f}")
    print(f"Recall:    {dm.get('recall', 0):.3f}")
    print(f"F1 score:  {dm.get('f1', 0):.3f}")
    print(f"Accuracy:  {dm.get('accuracy', 0):.3f}")
    print()
    print(f"TP={dm.get('TP')}, FP={dm.get('FP')}, FN={dm.get('FN')}, TN={dm.get('TN')}")
else:
    print("Napaka: open_world['decision_metrics'] je tipa", type(dm))
    print(dm)


Precision: 1.000
Recall:    1.000
F1 score:  1.000
Accuracy:  1.000

TP=7, FP=0, FN=0, TN=0


In [67]:
# Robust JSON export (handles Ellipsis, set/tuple, NaN, numpy/pandas types)
import json, datetime
from pathlib import Path

def to_jsonable(obj):
    # 1) direct fixes
    if obj is Ellipsis:
        return None
    # numpy scalars -> Python
    try:
        import numpy as np
        if isinstance(obj, (np.generic,)):
            return obj.item()
    except Exception:
        pass
    # pandas timestamps -> isoformat
    try:
        import pandas as pd
        if isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        # pandas NA/NaT/NaN -> None
        try:
            if pd.isna(obj):
                return None
        except Exception:
            pass
    except Exception:
        pass

    # 2) recursive structures
    if isinstance(obj, dict):
        return {k: to_jsonable(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [to_jsonable(x) for x in obj]
    if isinstance(obj, tuple):
        return [to_jsonable(x) for x in obj]
    if isinstance(obj, set):
        # sort for stability; also sanitize each element
        return sorted([to_jsonable(x) for x in obj], key=lambda x: str(x))

    # 3) everything else stays as-is (str, int, float, bool, None)
    return obj

OUT_JSON = "notebook_velenje/results_eval/eval_open_world_results.json"
Path(OUT_JSON).parent.mkdir(parents=True, exist_ok=True)

out = {
    "params": {
        "gold_json": GOLD_JSON,
        "preds_json": PREDS_JSON,
        "top_k": TOP_K,
        "threshold": THRESHOLD,
        "score_fields": SCORE_FIELDS,
        "timestamp": datetime.datetime.now().isoformat()
    },
    "open_world": open_world,
    "closed_world": closed_metrics
}

with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(to_jsonable(out), f, ensure_ascii=False, indent=2)

print("Saved JSON:", OUT_JSON)



Saved JSON: notebook_velenje/results_eval/eval_open_world_results.json
